In [ ]:
pip install pymysql

In [ ]:
import pandas as pd
import pymysql
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
low_memory=False

# Data is loaded into dataframe named df #

In [ ]:
# Step 1a: Python for Data Processing
#load data
df = pd.read_csv('police.csv')

In [ ]:
df

In [ ]:
df.info()

### Data has missing values in column search_type ###

In [ ]:
# Step 1b: Data Cleaning
# For dropping columns with all missing values
df.dropna(axis=1, how='all', inplace=True)
df.info()

In [ ]:
# filling na values by using driver_age column by introducing with median values as age for the new unknown data of search type
df.fillna({
    'driver_age': df['driver_age'].median(),
    'search_type': 'None',
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
}, inplace=True)

In [ ]:
df.info()

### Missing values filled ###

In [ ]:
# Concatenating date & time to create a new column named time_log
df['time_log'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])

In [ ]:
df.head(1)

In [ ]:
df.info()

In [ ]:
# Connect to MySQL
myconnection = pymysql.connect(host="localhost",user="root",password="Veerabhagu@00")

In [ ]:
cursor = myconnection.cursor()
cursor.execute("create database Securecheck")

In [ ]:
cursor.execute("USE Securecheck;")

In [ ]:
# create table police_log...using "if not exists" to prevent error from duplicate table creation...not using _raw columns from the data
cursor.execute("""CREATE TABLE IF NOT EXISTS police_log (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(50),
    time_log DATETIME
)""")

In [ ]:
df

### Date and time column should be in correct format. Even though the data seems correct, to make sure everything is correct, convert them once ###

In [ ]:
# Convert stop_date to correct format (YYYY-MM-DD)
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%Y-%m-%d') 
# coerce ensures that any invalid date entries are replaced with Not a Time by avoiding errors.
# dt.strftime('%Y-%m-%d')to format datetime values into a string

# convert stop_time to correct format (HH:MM:SS)
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')


In [ ]:
df

In [ ]:
# Replace NaN with None, df.where is used to fill none, instead of df.fillna, as fillna cannot fill none as a value
df = df.where(pd.notnull(df), None)

In [ ]:
df.head(2)

In [ ]:
# Insert Query
query = """
    INSERT INTO police_log (
        stop_date, stop_time, country_name, driver_gender, driver_age, driver_race,
        violation, search_conducted, search_type, stop_outcome, is_arrested,
        stop_duration, drugs_related_stop, vehicle_number, time_log
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Iterating DataFrame and Inserting rows
for i, row in df.iterrows():
    cursor.execute(query, (
    row.get('stop_date'),
    row.get('stop_time'),
    row.get('country_name'),
    row.get('driver_gender'),
    int(row.get('driver_age')) if row.get('driver_age') else None,
    row.get('driver_race'),
    row.get('violation'),
    bool(row.get('search_conducted')) if row.get('search_conducted') is not None else None,
    row.get('search_type'),
    row.get('stop_outcome'),
    bool(row.get('is_arrested')) if row.get ('is_arrested') is not None else None,
    row.get('stop_duration'),
    bool(row.get('drugs_related_stop')) if row.get('drugs_related_stop') is not None else None,
    row.get('vehicle_number'),
    row.get('time_log')
))

In [ ]:
# Commit and Close Connection
myconnection.commit()
cursor.close()
myconnection.close()